# REMOVE SPECTOGRAMS FROM .WAV

In [4]:
import os
import shutil

def remove_spectrograms():
    """
    Remove the processed spectrograms and associated folders.
    """
    processed_path = './music_data/musdb18hq_processed'

    if os.path.exists(processed_path):
        print(f'Removing path {processed_path} from OS and all of its contents.')
        shutil.rmtree(processed_path)
        print('Folder removed successfully.')
    else:
        print(f'No such path exists: {processed_path}')

remove_spectrograms()

Removing path ./music_data/musdb18hq_processed from OS and all of its contents.
Folder removed successfully.


# CREATE SPECTROGRAMS FROM .WAV

In [5]:
import os
import shutil
import torch
import torchaudio

def create_spectrogram(wav_path: str) -> torch.Tensor:
    """
    Create a spectrogram from a WAV file.

    Args:
        wav_path (str): Path to the input WAV file.

    Returns:
        torch.Tensor: Spectrogram of the input WAV file.
    """
    waveform, sample_rate = torchaudio.load(wav_path)
    spectrogram = torchaudio.transforms.Spectrogram(n_fft=510)(waveform)
    return spectrogram


def stereo_to_mono(stereo_spectrogram: torch.Tensor) -> torch.Tensor:
    """
    Convert a stereo spectrogram to mono.

    Args:
        stereo_spectrogram (torch.Tensor): Input stereo spectrogram with shape (2, freq_bins, time_frames).

    Returns:
        torch.Tensor: Mono spectrogram with shape (freq_bins, time_frames).
    """
    left_channel = stereo_spectrogram[0, :, :]
    right_channel = stereo_spectrogram[1, :, :]
    
    # Calculate RMS energy for each channel
    left_rms = torch.sqrt(torch.mean(left_channel ** 2, dim=0))
    right_rms = torch.sqrt(torch.mean(right_channel ** 2, dim=0))
    
    # Mix channels using RMS values
    mono_spectrogram = 0.5 * (left_channel / left_rms + right_channel / right_rms)
    
    return mono_spectrogram


def create_destination_folders(processed_path: str):
    """
    Create destination folders for processed WAV files.

    Args:
        processed_path (str): Path to the base processed folder.
    """
    # Create the folder to gather the processed WAV files.
    print(f'Creating path {processed_path} in OS')
    os.makedirs(processed_path)
    
    print(f'Creating path {processed_path}/test in OS')
    os.makedirs(os.path.join(processed_path, 'test'))
    
    print(f'Creating path {processed_path}/train in OS')
    os.makedirs(os.path.join(processed_path, 'train'))
    
    print(f'Creating path {processed_path}/test_spectrograms in OS')
    os.makedirs(os.path.join(processed_path, 'test_spectrograms'))
    
    print(f'Creating path {processed_path}/train_spectrograms in OS')
    os.makedirs(os.path.join(processed_path, 'train_spectrograms'))


def create_spectograms_from_wav():
    processed_path = './music_data/musdb18hq_processed'

    if os.path.exists(processed_path):
        print(f'Path {processed_path} already exists. Remove before creating new spectograms')
    else:

        create_destination_folders(processed_path)

        for root_folder in ['train', 'test']:

            print(f'Processing folder: {root_folder}')

            # Source folder path
            source_folder = f'./music_data/musdb18hq/{root_folder}'

            # Destination folder path
            destination_folder = f'./music_data/musdb18hq_processed/{root_folder}'

            # Iterate through directories in the source folder
            for idx, dirname in enumerate(os.listdir(source_folder)):
                # Get the full path of the current directory
                source_dir_path = os.path.join(source_folder, dirname)
                
                # Check if it's a directory
                if os.path.isdir(source_dir_path):
                    # Generate new directory name with leading zeros
                    new_dirname = f'{str(idx+1).zfill(4)}'
                    
                    # Destination directory path
                    destination_dir_path = os.path.join(destination_folder, new_dirname)
                    
                    # Create the destination directory
                    os.makedirs(destination_dir_path, exist_ok=True)
                    
                    # Iterate through files in the current directory
                    for filename in ['mixture.wav', 'vocals.wav']:
                        # Full path of the source file
                        source_file_path = os.path.join(source_dir_path, filename)
                        
                        # Full path of the destination file
                        destination_file_path = os.path.join(destination_dir_path, filename)
                        
                        # Copy the file to the destination folder
                        shutil.copy2(source_file_path, destination_file_path)
                        
                        print(f'Copied {filename} from {dirname} to {new_dirname}')
                else:
                    print(f'Skipped {dirname} (it is not a directory)')

            # Iterate through the numbered folders
            for idx, dirname in enumerate(os.listdir(destination_folder)):
                # Get the full path of the current directory
                folder_path = os.path.join(destination_folder, dirname)
                
                # Check if it's a directory
                if os.path.isdir(folder_path):
                    # File paths for mixture.wav and vocals.wav
                    mixture_path = os.path.join(folder_path, 'mixture.wav')
                    vocals_path = os.path.join(folder_path, 'vocals.wav')
                    
                    # Create spectrogram for mixture.wav
                    mixture_spectrogram = stereo_to_mono(create_spectrogram(mixture_path))
                    mixture_spectrogram = torch.nan_to_num(mixture_spectrogram)
                    
                    # Create spectrogram for vocals.wav
                    vocals_spectrogram = stereo_to_mono(create_spectrogram(vocals_path))
                    vocals_spectrogram = torch.nan_to_num(vocals_spectrogram)

                    total_spectrogram = torch.stack((mixture_spectrogram, vocals_spectrogram))
                    
                    # Saving them as tensors:
                    torch.save(total_spectrogram, f'./music_data/musdb18hq_processed/{root_folder}_spectrograms/{dirname}.pt')
            
                    print(f'Spectrograms created for {dirname}')
            print('\nAudio processing complete.')


create_spectograms_from_wav()

Creating path ./music_data/musdb18hq_processed in OS
Creating path ./music_data/musdb18hq_processed/test in OS
Creating path ./music_data/musdb18hq_processed/train in OS
Creating path ./music_data/musdb18hq_processed/test_spectrograms in OS
Creating path ./music_data/musdb18hq_processed/train_spectrograms in OS
Processing folder: train
Copied mixture.wav from The Districts - Vermont to 0001
Copied vocals.wav from The Districts - Vermont to 0001
Copied mixture.wav from BigTroubles - Phantom to 0002
Copied vocals.wav from BigTroubles - Phantom to 0002
Copied mixture.wav from Music Delta - Rockabilly to 0003
Copied vocals.wav from Music Delta - Rockabilly to 0003
Copied mixture.wav from Johnny Lokke - Promises & Lies to 0004
Copied vocals.wav from Johnny Lokke - Promises & Lies to 0004
Copied mixture.wav from Hezekiah Jones - Borrowed Heart to 0005
Copied vocals.wav from Hezekiah Jones - Borrowed Heart to 0005
Copied mixture.wav from Steven Clark - Bounty to 0006
Copied vocals.wav from St